<a href="https://colab.research.google.com/github/ibitoladgr8/Machine-Learning/blob/main/Blood%20glucose%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


In [ ]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
train_df.shape

(1853, 508)

In [ ]:
def delete_columns_with_empty_values(df, threshold):
  columns_to_drop = df.columns[df.isna().sum() >= threshold]
  df = df.drop(columns=columns_to_drop)
  return df

train_df = delete_columns_with_empty_values(train_df, 0.8*train_df.shape[0])
test_df = test_df[[c for c in train_df.columns if c in test_df.columns]]


In [ ]:
train_df.shape

(177024, 364)

In [ ]:
#train_df['time'] = pd.to_datetime(train_df['time'], format='%H:%M:%S')
#train_df['time'] = train_df['time'].dt.time
train_df.dropna(subset=['bg+1:00'], inplace = True)

In [ ]:
train_df.dtypes[-300:]

,0
bg-0:50,float64
bg-0:45,float64
bg-0:40,float64
bg-0:35,float64
bg-0:30,float64
...,...
cals-0:15,float64
cals-0:10,float64
cals-0:05,float64
cals-0:00,float64


In [ ]:
train_df.head()

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,cals-0:40,cals-0:35,cals-0:30,cals-0:25,cals-0:20,cals-0:15,cals-0:10,cals-0:05,cals-0:00,bg+1:00
0,p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [ ]:
X = train_df.drop(['id','p_num','bg+1:00'], axis=1)
y = train_df['bg+1:00']


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Prepare features and target
features = train_df.drop(columns=['id', 'p_num', 'time', 'bg+1:00'])
target = train_df['bg+1:00']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train XGBoost model
model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")



MAE: 1.8362565672943856


In [ ]:
pred = model.predict(test_df.drop(columns=['id', 'p_num', 'time']))
output = pd.DataFrame({'id': test_df['id'], 'bg+1:00': pred})
output.to_csv('submission.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
features = train_df.drop(columns=['id', 'p_num', 'time', 'bg+1:00'])
target = train_df['bg+1:00']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
modelrf = RandomForestRegressor(n_estimators=100, random_state=42)
modelrf.fit(X_train, y_train)

# Evaluate
y_pred = modelrf.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")

MAE: 1.8588872416891284


In [ ]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 100 and
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num', SimpleImputer(strategy='constant'),
                                 ['bg-5:55', 'bg-5:50', 'bg-5:45', 'bg-5:40',
                                  'bg-5:35', 'bg-5:30', 'bg-5:25', 'bg-5:20',
                                  'bg-5:15', 'bg-5:10', 'bg-5:05', 'bg-5:00',
                                  'bg-4:55', 'bg-4:50', 'bg-4:45', 'bg-4:40',
                                  'bg-4:35', 'bg-4:30', 'bg-4:25', 'bg-4:20',
                                  'bg-4:15', 'bg-4:10', 'bg-4:05', 'bg-4:00',
                                  'bg-3:55', 'bg-3:50', 'bg-3:45', 'bg-3...
                                  'activity-5:15', 'activity-5:10',
                                  'activity-5:05', 'activity-5:00',
                                  'activity-4:55', 'activity-4:50',
                                  'activity-4:45', 'activity-4:40',
                                  'activity-4:35', 'activity-4:30',
                                  'activity-4:25', 'activity-4:20',
                                  'activity-4:15', 'activity-4:10',
                                  'activity-4:05', 'activity-4:00',
                                  'activity-3:55', 'activity-3:50',
                                  'activity-3:45', 'activity-3:40',
                                  'activity-3:35', 'activity-3:30', ...])])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num', SimpleImputer(strategy='constant'),
                                 ['bg-5:55', 'bg-5:50', 'bg-5:45', 'bg-5:40',
                                  'bg-5:35', 'bg-5:30', 'bg-5:25', 'bg-5:20',
                                  'bg-5:15', 'bg-5:10', 'bg-5:05', 'bg-5:00',
                                  'bg-4:55', 'bg-4:50', 'bg-4:45', 'bg-4:40',
                                  'bg-4:35', 'bg-4:30', 'bg-4:25', 'bg-4:20',
                                  'bg-4:15', 'bg-4:10', 'bg-4:05', 'bg-4:00',
                                  'bg-3:55', 'bg-3:50', 'bg-3:45', 'bg-3...
                                  'activity-5:15', 'activity-5:10',
                                  'activity-5:05', 'activity-5:00',
                                  'activity-4:55', 'activity-4:50',
                                  'activity-4:45', 'activity-4:40',
                                  'activity-4:35', 'activity-4:30',
                                  'activity-4:25', 'activity-4:20',
                                  'activity-4:15', 'activity-4:10',
                                  'activity-4:05', 'activity-4:00',
                                  'activity-3:55', 'activity-3:50',
                                  'activity-3:45', 'activity-3:40',
                                  'activity-3:35', 'activity-3:30', ...])])

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2


In [ ]:
feature_names = preprocessor.get_feature_names_out()

# Calculate the number of features after preprocessing
num_features_after_preprocessing = len(feature_names)

# Update the input shape of your model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=[num_features_after_preprocessing]), # Updated input shape
    tf.keras.layers.Dropout(0.2),
    layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='relu')
])

# Re-compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
from sklearn.metrics import mean_absolute_error

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model
my_pipeline.named_steps['model'].fit(
    my_pipeline.named_steps['preprocessor'].transform(X_train),
    y_train,
    batch_size=512,  # Add batch size
    epochs=1000,      # Add epochs
    callbacks=[early_stopping],  # Add callbacks
    verbose=0         # Add verbose
)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
MAE: 5.843112796194413


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

model1=RandomForestRegressor(n_estimators=100, random_state=42)
my_pipeline1 = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model1)
                             ])

# Preprocessing of training data, fit model
#my_pipeline.named_steps['model'].fit(my_pipeline.named_steps['preprocessor'].transform(X_train), y_train, epochs=10)
# Preprocessing of validation data, get predictions
my_pipeline1.fit(X_train, y_train)
preds1 = my_pipeline1.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

predictions1=my_pipeline1.predict(test_df)
output = pd.DataFrame({'id': test_df['id'], 'bg+1:00': predictions1.flatten()})
output.to_csv('submissionrf.csv', index=False)

MAE: 5.843112796194413


In [ ]:
test_df.shape

(3644, 507)

In [ ]:
from xgboost import XGBRFRegressor

model2= XGBRFRegressor(n_estimators=100,
                      random_state=42,
                      learning_rate=0.05,
                      max_depth=3,
                      subsample=0.8,
                      colsample_bytree=0.8
                      )
my_pipeline2 = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model2)
                             ])

# Preprocessing of training data, fit model
#my_pipeline.named_steps['model'].fit(my_pipeline.named_steps['preprocessor'].transform(X_train), y_train, epochs=10)
# Preprocessing of validation data, get predictions
my_pipeline2.fit(X_train, y_train)
preds2 = my_pipeline2.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

predictions2=my_pipeline2.predict(test_df)
output = pd.DataFrame({'id': test_df['id'], 'bg+1:00': predictions2.flatten()})
output.to_csv('submissionxgb.csv', index=False)

MAE: 5.843112796194413
